In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as pypl
import seaborn as sd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
country_df = pd.read_csv('../input/india-odi-history/countries_list.csv')
country_df = country_df.drop(country_df.columns[0],axis=1)
country_df.head()

In [ ]:
cricket_df = pd.read_csv('../input/india-odi-history/file_cricket.csv')
cricket_df = cricket_df.drop(cricket_df.columns[:2],axis=1)
cricket_df.Date = pd.to_datetime(cricket_df.Date)
cricket_df['Month'] = cricket_df.Date.dt.month
cricket_df['Year'] = cricket_df.Date.dt.year
cricket_df.head()

In [ ]:
cricket_df.shape

In [ ]:
country_df.shape

In [ ]:
cricket_df.isnull().sum()

# The ground that India played the most in

In [ ]:
cricket_df['Ground'].value_counts()

is Sharjah Cricket Stadium.

# What matches India won and India lost

In [ ]:
def won_or_lost(result):
    
    winner = re.sub(r'\swon.*','',result)
    
    if winner == 'No Result':
        return 2
    elif winner != 'India':
        return 0
    elif winner == 'India':
        return 1

cricket_df['verdict'] = cricket_df['result'].apply(lambda x : won_or_lost(x))

In [ ]:
print(f"No. of matches India won : {cricket_df['verdict'][cricket_df['verdict'] == 1].count()}")
print(f"No. of matches India lost : {cricket_df['verdict'][cricket_df['verdict'] == 0].count()}")

# Which 5 year Period India saw huge win percent?

In [ ]:
fiveyear = pd.DataFrame(cricket_df.resample('5A',on='Date').verdict.mean()*100)
fiveyear['verdict'] = fiveyear['verdict'].apply(lambda x: round(x,2))
fiveyear

2014 - 2019 is the only time India saw a huge spike in win percent

# Which Ground managed to produce maximum wins to India?

In [ ]:
wins = cricket_df[cricket_df['verdict'] == 1]
wins_df = cricket_df.groupby(by='Ground').sum()
wins_df.sort_values(by='verdict',ascending=False)

Sharjah Cricket Stadium is the ground which had the most wins by India

# Create additional features like month, Year and find out which month/Year India has maximum number of wins?

In [ ]:
wins2_df = cricket_df[cricket_df['verdict'] == 1]
month_df = wins2_df.groupby(by='Month',as_index=False)['verdict'].sum().sort_values(by="verdict",ascending=False).reset_index(drop=True)
month_df

In [ ]:
year_df = wins2_df.groupby(by='Year').sum().drop('Month',axis=1)
year_df.sort_values(by="verdict",ascending=False)

October is the month and 1998 is the year that India won the most.

# Which ground has max winning chances if India wants to play against Australia/ England?

In [ ]:
cricket_df['against'] = cricket_df['Series'].apply(lambda x : re.sub(r'.*\(v.\s|\)|\d+\-\d+\s|(\sv.\sIndia|India\sv.\s)|\d+\s|','',x))
cricket_df['against'].unique()

In [ ]:
ocecric_df = cricket_df[(cricket_df['against'] == "Australia") | (cricket_df['against'] == "England")]
ocecric_df = cricket_df[(cricket_df['against'] != 2)]

In [ ]:
ods = ocecric_df.groupby(by=['Ground','against'],as_index=False).agg(['sum','count']).drop(['Month','Year','Series','result'],axis=1).reset_index()
ods.columns = list(map('_'.join,ods.columns.values))
ods['winning_chance'] = (ods['verdict_sum']/ods['verdict_count']*100)

In [ ]:
ods[(ods['against_'] == 'Australia') & ((ods['winning_chance'] < 100) & (ods['winning_chance'] > 0))].sort_values(by="winning_chance",ascending=False)

In [ ]:
ods[(ods['against_'] == 'England') & ((ods['winning_chance'] < 100) & (ods['winning_chance'] > 0))].sort_values(by="winning_chance",ascending=False)

# Which foreign country has maximum wins against India?

In [ ]:
lost_df = cricket_df[cricket_df['verdict'] == 0]
lost_df.groupby(by='against',as_index=False)['verdict'].count().sort_values(by='verdict',ascending=False)

# How did India Perform in ODI after 2011 world cup?

In [ ]:
wc_df = cricket_df[cricket_df['Date'] > '2011-04-02'].drop(['Month'],axis=1)

india_wins = pd.DataFrame(wc_df[wc_df['verdict'] == 1].groupby(by='Year',as_index=False)['verdict'].count())
india_lose = pd.DataFrame(wc_df[wc_df['verdict'] == 0].groupby(by='Year',as_index=False)['verdict'].count())

india_wins.columns = ['Year','wins']
india_lose.columns = ['Year','loss']

india_tally = pd.merge(india_wins,india_lose,on="Year")
india_tally

In [ ]:
pypl.plot(india_tally['Year'],india_tally['wins'])
pypl.plot(india_tally['Year'],india_tally['loss'])
pypl.legend(['wins','loss'])

From the graph, we can say that India's performance after the 2011 World Cup was better, with the exception of 2016 and 2020 (due to fewer ODI matches being played during the pandemic)